In [1]:
import pexpect
import time
import sys
import re
import datetime

In [2]:
#ENM3 -> 10.23.202.28 PW : "Enm#2021"
#ENM5 -> 10.167.30.28 PW : "Enm#2021"

child = pexpect.spawn('ssh -L 7898:localhost:8007 -J root@150.236.203.198 root@127.0.0.1 -p 8007', timeout=30, encoding='utf-8', logfile=sys.stdout)

child.maxread = 300000 

child.expect("root@150.236.203.198's password: ")
child.sendline('PMT@2022')
child.expect("root@127.0.0.1's password: ")
child.sendline('PMT@2022')

time.sleep(3)

# Shell command to take after the dates in modification dates
# child.sendline('find /var/opt/common5/mdt/ -type f -newermt 2023-04-28 -exec ls -l {} \;')

# Shell command to take only the folder
# child.sendline('find /var/opt/common5/mdt/20230205 -type f -exec ls -l {} \;')

# Shell command to take certain range dates in modification dates
child.sendline('find /var/opt/common5/mdt -type f -newermt 2023-06-01 ! -newermt 2023-07-01 -exec ls -l {} \;')

try:
    child.expect(pexpect.EOF, timeout=100)

except pexpect.TIMEOUT:
    child.close()
    raw = str(child.before)

root@150.236.203.198's password: PMT@2022

root@127.0.0.1's password: PMT@2022
find /var/opt/common5/mdt -type f -newermt 2023-06-01 ! -newermt 2023-07-01 -exec ls -l {} \;

Last login: Wed Jul  5 10:00:57 2023 from 192.168.21.6
! -newermt 2023-1:~(base) [root@pmt-xl-1 ~]# find /var/opt/common5/mdt -type f -newermt 2023-06-01  -07-01 -exec ls -l {} \;
-rw-r--r--. 1 root root 109709 Jun  7 11:34 /var/opt/common5/mdt/20230607/07/result_ue_cap.zip
-rw-r--r--. 1 root root 5216463 Jun  7 11:34 /var/opt/common5/mdt/20230607/07/ue_traffic.zip
-rw-r--r--. 1 root root 10668634 Jun  7 11:34 /var/opt/common5/mdt/20230607/07/ue_meas.zip
-rw-r--r--. 1 root root 7316622 Jun  7 11:34 /var/opt/common5/mdt/20230607/07/mdt_result.zip
-rw-r--r--. 1 root root 60995639 Jun  7 11:36 /var/opt/common5/mdt/20230607/07/ta_histogram.zip
-rw-r--r--. 1 root root 1122948 Jun  7 12:57 /var/opt/common5/mdt/20230607/08/result_ue_cap.zip
-rw-r--r--. 1 root root 43830314 Jun  7 12:57 /var/opt/common5/mdt/20230607/08/ue

In [ ]:
test1 = re.split(r'\s+|\n', raw)

In [ ]:
# Create list comprehension to filter only mdt_result.zip only 
# Option are ue_traffic or mdt_result

pattern = r'.*/(?!filter_)[^/]+/mdt_result\.zip$'
result = [path for path in test1 if re.match(pattern, path)]

In [ ]:
# Create list comprehension to generate weekend dates in June

weekend_dates = [date.strftime("%Y%m%d") for date in (datetime.date(2023, 6, day) for day in range(1, 31)) 
                 if date.weekday() >= 5]

# Create list comprehension to generate continous dates in June (from 25 June until 30 June)
continous_dates = re.compile(r'/var/opt/common5/mdt/202306(2[5-9]|30)/\d{2}/mdt_result\.zip')

In [ ]:
# Create list comprehension to filter June Month only
result = [path for path in result if '202306' in path]

# Create list comprehension to filter Out Some Date
result = [path for path in result if '20230601' not in path and '20230602' not in path] 

# Create list comprehension to filter out continous dates
result = [path for path in result if not continous_dates.search(path)]

# Create list comprehension to filter Out Weekend Date
result = [path for path in result if path.split('/')[5] not in weekend_dates]

# Create list comprehension to filter Busy Hour only
result = [path for path in result if '/07/' in path or '/08/' in path or '/09/' in path or '/17/' in path or '/18/' in path or '/19/' in path]

# Create list comprehension to filter Busy Hour only
# result = [path for path in result if '/11/' in path or '/12/' in path or '/13/' in path]

# result = [path for path in result if '/07/' in path or '/08/' in path or '/09/']

result.sort()

In [ ]:
with open(r'/home/nivag/location_to_dl_8007-mdt.txt', 'w') as fp:
    fp.write('\n'.join(result))